In [65]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import json
from pathlib import Path
import alpaca_trade_api as tradeapi
import requests
import os 
from dotenv import load_dotenv

%matplotlib inline


from backtesting.test import SMA




In [66]:
load_dotenv()

alpaca_api_key = os.getenv("Api_key")
alpaca_secret_key = os.getenv("Secret_Key")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')


type(alpaca_api_key)


str

In [67]:
xlf_df = api.alpha_vantage.historic_quotes('XLF', adjusted=True, output_format='pandas')
xlf_df.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2020-05-01,22.25,22.310,21.9115,22.06,22.06,37519690.0,0.0,1.0
2020-04-30,22.91,23.160,22.6750,22.79,22.79,56730669.0,0.0,1.0
2020-04-29,23.42,23.655,23.1800,23.38,23.38,70050222.0,0.0,1.0
2020-04-28,23.16,23.325,22.6700,22.72,22.72,46430248.0,0.0,1.0
2020-04-27,21.96,22.600,21.9200,22.48,22.48,109428235.0,0.0,1.0


In [68]:
xlf_df.columns

Index(['1. open', '2. high', '3. low', '4. close', '5. adjusted close',
       '6. volume', '7. dividend amount', '8. split coefficient'],
      dtype='object')

In [69]:
xlf_df.rename(columns={'1. open':'Open','2. high':'High','3. low':'Low', '5. adjusted close':'Close', '6. volume':'Volume'}, inplace=True)
xlf_df.drop(columns=['4. close','7. dividend amount', '8. split coefficient'], inplace=True)
xlf_df

,Open,High,Low,Close,Volume
date,,,,,
2020-05-01,22.25,22.310,21.9115,22.0600,37519690.0
2020-04-30,22.91,23.160,22.6750,22.7900,56730669.0
2020-04-29,23.42,23.655,23.1800,23.3800,70050222.0
2020-04-28,23.16,23.325,22.6700,22.7200,46430248.0
2020-04-27,21.96,22.600,21.9200,22.4800,109428235.0
...,...,...,...,...,...
2000-05-05,22.69,23.160,22.6600,12.4970,60500.0
2000-05-04,23.13,23.190,22.7800,12.4158,146900.0
2000-05-03,23.78,23.980,23.0900,12.6106,345500.0


In [70]:
xlf_df.sort_index(ascending=True, inplace=True)
xlf_df

,Open,High,Low,Close,Volume
date,,,,,
2000-05-01,23.38,24.280,23.3400,12.9624,95100.0
2000-05-02,23.69,23.940,23.6400,12.8380,145800.0
2000-05-03,23.78,23.980,23.0900,12.6106,345500.0
2000-05-04,23.13,23.190,22.7800,12.4158,146900.0
2000-05-05,22.69,23.160,22.6600,12.4970,60500.0
...,...,...,...,...,...
2020-04-27,21.96,22.600,21.9200,22.4800,109428235.0
2020-04-28,23.16,23.325,22.6700,22.7200,46430248.0
2020-04-29,23.42,23.655,23.1800,23.3800,70050222.0


In [74]:
xlf_df.drop(columns=["Volume"], inplace=True)

In [75]:
class SmaCross(Strategy):
    def init(self):
        Close = self.data.Close
        self.ma1 = self.I(SMA, Close, 10)
        self.ma2 = self.I(SMA, Close, 20)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


bt = Backtest(xlf_df, SmaCross,
              cash=10000, commission=.002)
bt.run()


Start                     2000-05-01 00:00:00
End                       2020-05-01 00:00:00
Duration                   7305 days 00:00:00
Exposure [%]                          98.9733
Equity Final [$]                      1571.88
Equity Peak [$]                       14896.2
Return [%]                           -84.2812
Buy & Hold Return [%]                 70.1845
Max. Drawdown [%]                    -91.0972
Avg. Drawdown [%]                     -52.713
Max. Drawdown Duration     7156 days 00:00:00
Avg. Drawdown Duration     2418 days 00:00:00
# Trades                                  277
Win Rate [%]                           34.657
Best Trade [%]                        47.0293
Worst Trade [%]                      -22.7831
Avg. Trade [%]                      -0.482892
Max. Trade Duration         128 days 00:00:00
Avg. Trade Duration          27 days 00:00:00
Expectancy [%]                        4.72939
SQN                                  -1.89957
Sharpe Ratio                      

In [76]:
bt.plot()